# Описание проекта

23 задачи на базовые запросы SQL


1.
Посчитайте, сколько компаний закрылось.

SELECT COUNT (id)
FROM company
WHERE status = 'closed';

2.
Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

SELECT SUM(funding_total) AS ft
FROM company
WHERE category_code = 'news'
AND country_code ='USA'
GROUP BY id
ORDER BY ft DESC;

3.
Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

SELECT SUM(price_amount)
FROM acquisition
WHERE EXTRACT (YEAR FROM CAST (acquired_at AS date)) IN (2011,2012,2013)
AND term_code = 'cash';

4.
Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%'
ORDER BY first_name;

5.
Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%'
ORDER BY id;

6.
Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

SELECT country_code, 
       SUM (funding_total) AS ft
FROM company
GROUP BY country_code
ORDER BY ft DESC;

7.
Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

SELECT CAST(funded_at AS date),
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY CAST (funded_at AS date)
HAVING MIN(raised_amount) <> MAX(raised_amount)
AND MIN(raised_amount) <> 0
ORDER BY CAST (funded_at AS date);

8.
Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.



In [ ]:
8.
Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

SELECT * ,
     CASE 
         WHEN invested_companies >= 100 THEN 'high_activity'
         WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
         ELSE 'low_activity'
      END activity
FROM fund 
ORDER BY name;

9.
Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

SELECT ROUND (AVG (investment_rounds)) AS avg,
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity
FROM fund
GROUP BY activity
ORDER BY avg;

10.
Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
SELECT DISTINCT country_code,
       AVG (invested_companies),
       MIN (invested_companies),
       MAX (invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST (founded_at AS date)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN (invested_companies) <> 0
ORDER BY AVG (invested_companies)DESC, country_code
LIMIT 10;

11.
Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT first_name,
       last_name,
       instituition
FROM people AS p
LEFT JOIN education AS ed ON p.id = ed.person_id;

12.
Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

SELECT c.name,
       COUNT (DISTINCT ed.instituition) AS ed_count
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
LEFT JOIN education AS ed ON p.id = ed.person_id
GROUP BY c.id
ORDER BY ed_count DESC
LIMIT 5;

13.
Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

SELECT DISTINCT c.name
FROM company AS c 
INNER JOIN funding_round AS fr ON c.id=fr.company_id
WHERE is_first_round = 1
AND is_last_round = 1
AND status= 'closed'
GROUP BY c.name;


14.
Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

SELECT dead_comp.ppl
FROM (SELECT DISTINCT c.name,
      p.id as ppl
FROM company AS c 
INNER JOIN funding_round AS fr ON c.id=fr.company_id
INNER JOIN people AS p ON c.id = p.company_id
WHERE is_first_round = 1
AND is_last_round = 1
AND status= 'closed') AS dead_comp;


15.
Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT dead_comp.ppl,
       dead_comp.inst
FROM (SELECT DISTINCT c.name,
      p.id as ppl,
      instituition AS inst
FROM company AS c 
INNER JOIN funding_round AS fr ON c.id=fr.company_id
INNER JOIN people AS p ON c.id = p.company_id
INNER JOIN education AS ed ON p.id = ed.person_id
WHERE is_first_round = 1
AND is_last_round = 1
AND status= 'closed') AS dead_comp;


16.
Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

SELECT dead_comp.ppl,
       COUNT (instituition)
FROM (SELECT DISTINCT c.name,
      p.id as ppl
FROM company AS c 
INNER JOIN funding_round AS fr ON c.id=fr.company_id
INNER JOIN people AS p ON c.id = p.company_id
WHERE is_first_round = 1
AND is_last_round = 1
AND status= 'closed') AS dead_comp
INNER JOIN education AS ed ON dead_comp.ppl = ed.person_id
GROUP BY dead_comp.ppl;

17.
Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT AVG(filter.count)
FROM (SELECT dead_comp.ppl,
       COUNT (instituition) AS count
FROM (SELECT DISTINCT c.name,
      p.id as ppl
FROM company AS c 
INNER JOIN funding_round AS fr ON c.id=fr.company_id
INNER JOIN people AS p ON c.id = p.company_id
WHERE is_first_round = 1
AND is_last_round = 1
AND status= 'closed') AS dead_comp
INNER JOIN education AS ed ON dead_comp.ppl = ed.person_id
GROUP BY dead_comp.ppl) AS filter;


18.
Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
SELECT AVG(filter.count)
FROM (SELECT fb_comp.ppl,
       COUNT (instituition) AS count
FROM (SELECT DISTINCT c.name,
      p.id as ppl
FROM company AS c 
INNER JOIN funding_round AS fr ON c.id=fr.company_id
INNER JOIN people AS p ON c.id = p.company_id
WHERE c.name = 'Facebook') AS fb_comp
INNER JOIN education AS ed ON fb_comp.ppl = ed.person_id
GROUP BY fb_comp.ppl) AS filter;


19.
Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

SELECT f.name AS name_of_fund, 
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i JOIN company AS c ON i.company_id = c.id
JOIN fund AS f ON f.id = i.fund_id
JOIN funding_round AS fr ON fr.id = i.funding_round_id
WHERE c.milestones > 6
  AND EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN 2012 AND 2013

20.
Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

SELECT a_ing.name, 
       a_ing.price_amount,
       a_ed.name,
       a_ed.funding_total,
       ROUND(a_ing.price_amount / a_ed.funding_total) AS income_share
FROM (SELECT comp.name,
             acq.id,
             acq.acquiring_company_id,
             acq.price_amount
             FROM acquisition AS acq
             LEFT JOIN company AS comp ON acq.acquiring_company_id = comp.id
            ) AS a_ing 
LEFT JOIN (SELECT comp.name,
                  acq.id,
                  acq.acquired_company_id,
                  comp.funding_total
            FROM acquisition AS acq
            LEFT JOIN company AS comp ON acq.acquired_company_id = comp.id
            ) AS a_ed ON a_ing.id = a_ed.id
WHERE a_ing.price_amount <> 0
AND a_ed.funding_total <> 0
GROUP BY a_ing.name, a_ing.price_amount, a_ed.name, a_ed.funding_total
ORDER BY a_ing.price_amount DESC,  a_ed.name
LIMIT 10;

       

21.
Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name,
       EXTRACT (MONTH FROM CAST(funded_at AS date))
FROM company AS c
INNER JOIN funding_round AS fr ON c.id = fr.company_id
WHERE category_code = 'social'
AND raised_amount <> 0
AND EXTRACT (YEAR FROM CAST(funded_at AS date)) BETWEEN 2010 AND 2013
ORDER BY c.name;

22.
Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.

WITH
a AS (SELECT EXTRACT (MONTH FROM CAST(funded_at AS date)) AS month,
       id,
       company_id,
       raised_amount
      FROM funding_round AS fr
      WHERE EXTRACT (YEAR FROM CAST(funded_at AS date)) BETWEEN 2010 AND 2013),

b AS (SELECT DISTINCT name AS name,
      f.id,
      inv.funding_round_id AS fr_id
      FROM fund AS f
      LEFT JOIN investment AS inv ON inv.fund_id = f.id
      WHERE country_code = 'USA'),

c AS (SELECT acquired_company_id,
             comp.id AS c_id,
             EXTRACT (MONTH FROM CAST(acquired_at AS date)) AS month
      FROM acquisition AS acq
      LEFT JOIN company AS comp ON acq.acquired_company_id = comp.id)

SELECT a.month,
       b.name,
       COUNT(c.acquired_company_id) AS count_id,
       SUM (a.raised_amount) AS sum_tot
FROM a
INNER JOIN b ON a.id = b.fr_id
INNER JOIN c ON a.month = c.month
GROUP BY a.month, b.name;

23.
Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
a AS(SELECT country_code, 
       AVG(funding_total) AS year_2011
FROM company
WHERE EXTRACT(YEAR FROM CAST (founded_at AS date)) = 2011
GROUP BY country_code),

b AS(SELECT country_code, 
       AVG(funding_total) AS year_2012
FROM company
WHERE EXTRACT(YEAR FROM CAST (founded_at AS date)) = 2012
GROUP BY country_code),

c AS(SELECT country_code, 
       AVG(funding_total) AS year_2013
FROM company
WHERE EXTRACT(YEAR FROM CAST (founded_at AS date)) = 2013
GROUP BY country_code)

SELECT a.country_code,
       year_2011,
       year_2012,
       year_2013
FROM a 
JOIN b ON a.country_code = b.country_code
JOIN c ON a.country_code = c.country_code
ORDER BY year_2011 DESC